In [ ]:

import zipfile, os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

# Krok 1. Rozpakuj ZIP
zip_path = "/content/tissue_flow_ai_demo_final_ready_AUTOMATIC.zip"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/")

# Krok 2. Ścieżki
base_dir = "/content/tissue_flow_ai_demo_final_ready/images"
augmented_dir = "/content/augmented_images"

# Krok 3. Augmentacja
augmentor = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.15,
    height_shift_range=0.15,
    horizontal_flip=True,
    brightness_range=(0.8, 1.2),
    fill_mode='nearest'
)

copies_per_image = 15
if not os.path.exists(augmented_dir):
    os.makedirs(augmented_dir)
    for class_dir in os.listdir(base_dir):
        os.makedirs(os.path.join(augmented_dir, class_dir), exist_ok=True)

    for class_dir in os.listdir(base_dir):
        class_input_path = os.path.join(base_dir, class_dir)
        class_output_path = os.path.join(augmented_dir, class_dir)
        for img_file in os.listdir(class_input_path):
            img_path = os.path.join(class_input_path, img_file)
            img = load_img(img_path)
            x = img_to_array(img)
            x = x.reshape((1,) + x.shape)
            i = 0
            for batch in augmentor.flow(x, batch_size=1, save_to_dir=class_output_path, save_prefix='aug', save_format='png'):
                i += 1
                if i >= copies_per_image:
                    break

# Krok 4. Wczytanie danych
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.4)

train_data = datagen.flow_from_directory(
    augmented_dir, target_size=(224, 224), batch_size=4,
    class_mode='binary', subset='training', shuffle=True)

val_data = datagen.flow_from_directory(
    augmented_dir, target_size=(224, 224), batch_size=4,
    class_mode='binary', subset='validation', shuffle=False)

print(f"Liczba próbek treningowych: {train_data.samples}")
print(f"Liczba próbek walidacyjnych: {val_data.samples}")

# Krok 5. Budowa modelu
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_data, validation_data=val_data, epochs=5)

# Krok 6. Fine-tuning
base_model.trainable = True
for layer in base_model.layers[:120]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
fine_tune_history = model.fit(train_data, validation_data=val_data, epochs=5)

# Krok 7. Wykresy
plt.plot(fine_tune_history.history['accuracy'], label='Train Accuracy (FT)')
plt.plot(fine_tune_history.history['val_accuracy'], label='Val Accuracy (FT)')
plt.title('Dokładność po fine-tuningu')
plt.legend()
plt.grid(True)
plt.show()

plt.plot(fine_tune_history.history['loss'], label='Train Loss (FT)')
plt.plot(fine_tune_history.history['val_loss'], label='Val Loss (FT)')
plt.title('Strata po fine-tuningu')
plt.legend()
plt.grid(True)
plt.show()
